Reference: https://github.com/SuYenTing/Python-web-crawler

# 載入套件

In [1]:
import requests
import pandas as pd
import time
import os
import re
from bs4 import BeautifulSoup
import json
import datetime

# 參數設定

In [2]:
# 欲下載新聞的股票關鍵字清單
list_0050 = '鴻海	, 聯發科	, 台達電	, 聯電	, 中華電	, 南亞	, 富邦金	, 中信金	, 中鋼	, 兆豐金	, 玉山金	, 統一	, 台塑	, 國泰金	, 日月光投控	, 第一金	, 合庫金	, 台化	, 元大金	, 中租-KY	, 華南金	, 台泥	, 廣達	, 開發金	, 和泰車	, 大立光	, 永豐金	, 華碩	, 台灣大	, 統一超	, 上海商銀	, 台新金	, 欣興	, 聯詠	, 長榮	, 國巨	, 臺股期貨	, 友達	, 矽力*-KY	, 研華	, 瑞昱	, 亞德客-KY	, 彰銀	, 遠傳	, 陽明	, 台塑化	, 萬海	, 豐泰	, 南亞科	, 南電	,'
list_0051 = "智邦\
,華新\
,光寶科\
,遠東新\
,可成\
,和碩\
,新光金\
,亞泥\
,群創\
,仁寶\
,旭隼\
,力積電\
,儒鴻\
,緯創\
,微星\
聯強,\
大同,\
長榮航,\
致茂,\
英業達,\
緯穎,\
大聯大,\
臻鼎-KY,\
臺企銀,\
正新,\
宏碁,\
巨大,\
世芯-KY,\
大成鋼,\
台灣高鐵,\
華邦電,\
旺宏,\
華航,\
嘉澤,\
信邦,\
東元,\
力成,\
鴻準,\
上銀,\
興富發,\
群光,\
華通,\
創意,\
台光電,\
健鼎,\
瑞儀,\
美利達,\
台中銀,\
技嘉,\
潤泰新\
台肥,\
宏達電,\
寶成,\
貿聯-KY,\
聚陽,\
佳世達,\
華新科,\
健策,\
聯邦銀,\
億豐,\
國票金,\
京元電子,\
裕融,\
北極星藥業-KY,\
京城銀,\
大成,\
永豐餘,\
長興,\
東和鋼鐵,\
川湖,\
遠東銀,\
玉晶光,\
智原,\
台玻,\
金像電,\
中保科,\
富邦媒,\
福懋,\
同欣電,\
富采,\
祥碩,\
景碩,\
慧洋-KY,\
文曄,\
采鈺,\
愛普,\
裕隆,\
潤泰全,\
中鴻,\
新唐,\
AES-KY,\
遠雄,\
和潤企業,\
裕民,\
美時,\
超豐,\
台勝科,\
裕日車,\
燁輝"

In [3]:
search_0050 = list_0050.strip('\t,').split('\t, ')
search_0051 = list_0051.strip(',').split(',')
QueryList = search_0050 + search_0051

QueryList.append('台灣50')
QueryList.append('0050')
QueryList.append('0051')
# 新聞下載起始日
StartDate = (datetime.date.today() + datetime.timedelta(days=-7)).strftime('%Y-%m-%d')
StartDate

'2022-12-17'

In [4]:
print(datetime.datetime.now())

2022-12-24 16:43:26.095956


In [5]:
searchList_all = []
sz = len(QueryList)
terms = int(sz/20)+1
for i in range(terms):
    searchList_all.append(QueryList[20*i:20*(i+1)])

# 相關函數

In [6]:
# 整理Google新聞資料用
def arrangeGoogleNews(elem):
    return ([elem.find('title').getText(),
             elem.find('link').getText(),
             elem.find('pubDate').getText(),
             elem.find('description').getText(),
             elem.find('source').getText()])

In [7]:
# 擷取各家新聞網站新聞函數
# First part
def beautifulSoupNews(url):
    
    # 設定hearers
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/87.0.4280.141 Safari/537.36'}

    # 下載網站原始碼
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        transfer_url = soup.find("meta", attrs={"property": "og:url"})['content']
        # 判斷url網域做對應文章擷取
        domain = re.findall('https://[^/]*', transfer_url)[0].replace('https://', '')
    except: 
        domain = "unknow"

    if domain == 'udn.com':
        try:
            # 聯合新聞網
            item = soup.find_all('section', class_='article-content__editor')[0].find_all('p')
            content = [elem.getText() for elem in item]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ')
        except:
            content = 'unknow domain'
            pass

    elif domain in ['tw.stock.yahoo.com', 'tw.news.yahoo.com']:
        try:
            # Yahoo奇摩
            item = soup.find_all('div', class_='caas-body')[0].find_all('p')
            content = [elem.getText() for elem in item]
            del_text = soup.find_all('div', class_='caas-body')[0].find_all('a')
            del_text = [elem.getText() for elem in del_text]
            content = [elem for elem in content if elem not in del_text]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ').replace(u'\xa0', ' ')
        except:
            content = 'unknow domain'
            pass

    elif domain == 'money.udn.com':
        try:
            # 經濟日報
            item = soup.find_all('section', id='article_body')[0].find_all('p')
            content = [elem.getText() for elem in item]
            content = [elem for elem in content]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ')
        except:
            content = 'unknow domain'
            pass

    elif domain == 'www.chinatimes.com':
        try:
            # 中時新聞網
            item = soup.find_all('div', class_='article-body')[0].find_all('p')
            content = [elem.getText() for elem in item]
            content = [elem for elem in content]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ')
        except:
            content = 'unknow domain'
            pass

    elif domain == 'ctee.com.tw':
        try:
            # 工商時報
            item = soup.find_all('div', class_='entry-content clearfix single-post-content')[0].find_all('p')
            content = [elem.getText() for elem in item]
            content = [elem for elem in content]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ')
        except:
            content = 'unknow domain'
            pass

    elif domain == 'news.cnyes.com':
        try:
            # 鉅亨網
            item = soup.find_all('div', itemprop='articleBody')[0].find_all('p')
            content = [elem.getText() for elem in item]
            content = [elem for elem in content]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ').replace(u'\xa0', ' ')
        except:
            content = 'unknow domain'
            pass

    elif domain == 'finance.ettoday.net':
        try:
            # ETtoday
            item = soup.find_all('div', itemprop='articleBody')[0].find_all('p')
            content = [elem.getText() for elem in item]
            content = [elem for elem in content]
            content = ''.join(content)
            content = content.replace('\r', ' ').replace('\n', ' ').replace(u'\xa0', ' ')
        except:
            content = 'unknow domain'
            pass

    else:
        # 未知domain
        content = 'unknow domain'

    return content

# 迴圈下載股票清單的Google新聞資料

In [8]:
stockNews = pd.DataFrame()
searchList = searchList_all[0]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 鴻海 在Google的新聞清單  進度: 1 / 20
目前正在下載: 鴻海 各家新聞  進度: 1 / 35
目前正在下載: 鴻海 各家新聞  進度: 2 / 35
目前正在下載: 鴻海 各家新聞  進度: 3 / 35
目前正在下載: 鴻海 各家新聞  進度: 4 / 35
目前正在下載: 鴻海 各家新聞  進度: 5 / 35
目前正在下載: 鴻海 各家新聞  進度: 6 / 35
目前正在下載: 鴻海 各家新聞  進度: 7 / 35
目前正在下載: 鴻海 各家新聞  進度: 8 / 35
目前正在下載: 鴻海 各家新聞  進度: 9 / 35
目前正在下載: 鴻海 各家新聞  進度: 10 / 35
目前正在下載: 鴻海 各家新聞  進度: 11 / 35
目前正在下載: 鴻海 各家新聞  進度: 12 / 35
目前正在下載: 鴻海 各家新聞  進度: 13 / 35
目前正在下載: 鴻海 各家新聞  進度: 14 / 35
目前正在下載: 鴻海 各家新聞  進度: 15 / 35
目前正在下載: 鴻海 各家新聞  進度: 16 / 35
目前正在下載: 鴻海 各家新聞  進度: 17 / 35
目前正在下載: 鴻海 各家新聞  進度: 18 / 35
目前正在下載: 鴻海 各家新聞  進度: 19 / 35
目前正在下載: 鴻海 各家新聞  進度: 20 / 35
目前正在下載: 鴻海 各家新聞  進度: 21 / 35
目前正在下載: 鴻海 各家新聞  進度: 22 / 35
目前正在下載: 鴻海 各家新聞  進度: 23 / 35
目前正在下載: 鴻海 各家新聞  進度: 24 / 35
目前正在下載: 鴻海 各家新聞  進度: 25 / 35
目前正在下載: 鴻海 各家新聞  進度: 26 / 35
目前正在下載: 鴻海 各家新聞  進度: 27 / 35
目前正在下載: 鴻海 各家新聞  進度: 28 / 35
目前正在下載: 鴻海 各家新聞  進度: 29 / 35
目前正在下載: 鴻海 各家新聞  進度: 30 / 35
目前正在下載: 鴻海 各家新聞  進度: 31 / 35
目前正在下載: 鴻海 各家新聞  進度: 32 / 35
目前正在下載: 鴻海 各家新聞  進度: 33 / 35
目前正在下載: 鴻海 各家新

目前正在下載: 中鋼 各家新聞  進度: 8 / 35
目前正在下載: 中鋼 各家新聞  進度: 9 / 35
目前正在下載: 中鋼 各家新聞  進度: 10 / 35
目前正在下載: 中鋼 各家新聞  進度: 11 / 35
目前正在下載: 中鋼 各家新聞  進度: 12 / 35
目前正在下載: 中鋼 各家新聞  進度: 13 / 35
目前正在下載: 中鋼 各家新聞  進度: 14 / 35
目前正在下載: 中鋼 各家新聞  進度: 15 / 35
目前正在下載: 中鋼 各家新聞  進度: 16 / 35
目前正在下載: 中鋼 各家新聞  進度: 17 / 35
目前正在下載: 中鋼 各家新聞  進度: 18 / 35
目前正在下載: 中鋼 各家新聞  進度: 19 / 35
目前正在下載: 中鋼 各家新聞  進度: 20 / 35
目前正在下載: 中鋼 各家新聞  進度: 21 / 35
目前正在下載: 中鋼 各家新聞  進度: 22 / 35
目前正在下載: 中鋼 各家新聞  進度: 23 / 35
目前正在下載: 中鋼 各家新聞  進度: 24 / 35
目前正在下載: 中鋼 各家新聞  進度: 25 / 35
目前正在下載: 中鋼 各家新聞  進度: 26 / 35
目前正在下載: 中鋼 各家新聞  進度: 27 / 35
目前正在下載: 中鋼 各家新聞  進度: 28 / 35
目前正在下載: 中鋼 各家新聞  進度: 29 / 35
目前正在下載: 中鋼 各家新聞  進度: 30 / 35
目前正在下載: 中鋼 各家新聞  進度: 31 / 35
目前正在下載: 中鋼 各家新聞  進度: 32 / 35
目前正在下載: 中鋼 各家新聞  進度: 33 / 35
目前正在下載: 中鋼 各家新聞  進度: 34 / 35
目前正在下載: 中鋼 各家新聞  進度: 35 / 35
目前正在搜尋股票: 兆豐金 在Google的新聞清單  進度: 10 / 20
目前正在下載: 兆豐金 各家新聞  進度: 1 / 22
目前正在下載: 兆豐金 各家新聞  進度: 2 / 22
目前正在下載: 兆豐金 各家新聞  進度: 3 / 22
目前正在下載: 兆豐金 各家新聞  進度: 4 / 22
目前正在下載: 兆豐金 各家新聞  進度: 5 / 22
目前正在下

目前正在下載: 第一金 各家新聞  進度: 43 / 45
目前正在下載: 第一金 各家新聞  進度: 44 / 45
目前正在下載: 第一金 各家新聞  進度: 45 / 45
目前正在搜尋股票: 合庫金 在Google的新聞清單  進度: 17 / 20
目前正在下載: 合庫金 各家新聞  進度: 1 / 19
目前正在下載: 合庫金 各家新聞  進度: 2 / 19
目前正在下載: 合庫金 各家新聞  進度: 3 / 19
目前正在下載: 合庫金 各家新聞  進度: 4 / 19
目前正在下載: 合庫金 各家新聞  進度: 5 / 19
目前正在下載: 合庫金 各家新聞  進度: 6 / 19
目前正在下載: 合庫金 各家新聞  進度: 7 / 19
目前正在下載: 合庫金 各家新聞  進度: 8 / 19
目前正在下載: 合庫金 各家新聞  進度: 9 / 19
目前正在下載: 合庫金 各家新聞  進度: 10 / 19
目前正在下載: 合庫金 各家新聞  進度: 11 / 19
目前正在下載: 合庫金 各家新聞  進度: 12 / 19
目前正在下載: 合庫金 各家新聞  進度: 13 / 19
目前正在下載: 合庫金 各家新聞  進度: 14 / 19
目前正在下載: 合庫金 各家新聞  進度: 15 / 19
目前正在下載: 合庫金 各家新聞  進度: 16 / 19
目前正在下載: 合庫金 各家新聞  進度: 17 / 19
目前正在下載: 合庫金 各家新聞  進度: 18 / 19
目前正在下載: 合庫金 各家新聞  進度: 19 / 19
目前正在搜尋股票: 台化 在Google的新聞清單  進度: 18 / 20
目前正在下載: 台化 各家新聞  進度: 1 / 31
目前正在下載: 台化 各家新聞  進度: 2 / 31
目前正在下載: 台化 各家新聞  進度: 3 / 31
目前正在下載: 台化 各家新聞  進度: 4 / 31
目前正在下載: 台化 各家新聞  進度: 5 / 31
目前正在下載: 台化 各家新聞  進度: 6 / 31
目前正在下載: 台化 各家新聞  進度: 7 / 31
目前正在下載: 台化 各家新聞  進度: 8 / 31
目前正在下載: 台化 各家新聞  進度: 9 / 31
目前正在下載: 台化 各家新聞  進

In [9]:
searchList = searchList_all[1]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 華南金 在Google的新聞清單  進度: 1 / 20
目前正在下載: 華南金 各家新聞  進度: 1 / 20
目前正在下載: 華南金 各家新聞  進度: 2 / 20
目前正在下載: 華南金 各家新聞  進度: 3 / 20
目前正在下載: 華南金 各家新聞  進度: 4 / 20
目前正在下載: 華南金 各家新聞  進度: 5 / 20
目前正在下載: 華南金 各家新聞  進度: 6 / 20
目前正在下載: 華南金 各家新聞  進度: 7 / 20
目前正在下載: 華南金 各家新聞  進度: 8 / 20
目前正在下載: 華南金 各家新聞  進度: 9 / 20
目前正在下載: 華南金 各家新聞  進度: 10 / 20
目前正在下載: 華南金 各家新聞  進度: 11 / 20
目前正在下載: 華南金 各家新聞  進度: 12 / 20
目前正在下載: 華南金 各家新聞  進度: 13 / 20
目前正在下載: 華南金 各家新聞  進度: 14 / 20
目前正在下載: 華南金 各家新聞  進度: 15 / 20
目前正在下載: 華南金 各家新聞  進度: 16 / 20
目前正在下載: 華南金 各家新聞  進度: 17 / 20
目前正在下載: 華南金 各家新聞  進度: 18 / 20
目前正在下載: 華南金 各家新聞  進度: 19 / 20
目前正在下載: 華南金 各家新聞  進度: 20 / 20
目前正在搜尋股票: 台泥 在Google的新聞清單  進度: 2 / 20
目前正在下載: 台泥 各家新聞  進度: 1 / 29
目前正在下載: 台泥 各家新聞  進度: 2 / 29
目前正在下載: 台泥 各家新聞  進度: 3 / 29
目前正在下載: 台泥 各家新聞  進度: 4 / 29
目前正在下載: 台泥 各家新聞  進度: 5 / 29
目前正在下載: 台泥 各家新聞  進度: 6 / 29
目前正在下載: 台泥 各家新聞  進度: 7 / 29
目前正在下載: 台泥 各家新聞  進度: 8 / 29
目前正在下載: 台泥 各家新聞  進度: 9 / 29
目前正在下載: 台泥 各家新聞  進度: 10 / 29
目前正在下載: 台泥 各家新聞  進度: 11 / 29
目前正在下載: 台泥 各家新聞  進度: 1

目前正在下載: 台灣大 各家新聞  進度: 17 / 68
目前正在下載: 台灣大 各家新聞  進度: 18 / 68
目前正在下載: 台灣大 各家新聞  進度: 19 / 68
目前正在下載: 台灣大 各家新聞  進度: 20 / 68
目前正在下載: 台灣大 各家新聞  進度: 21 / 68
目前正在下載: 台灣大 各家新聞  進度: 22 / 68
目前正在下載: 台灣大 各家新聞  進度: 23 / 68
目前正在下載: 台灣大 各家新聞  進度: 24 / 68
目前正在下載: 台灣大 各家新聞  進度: 25 / 68
目前正在下載: 台灣大 各家新聞  進度: 26 / 68
目前正在下載: 台灣大 各家新聞  進度: 27 / 68
目前正在下載: 台灣大 各家新聞  進度: 28 / 68
目前正在下載: 台灣大 各家新聞  進度: 29 / 68
目前正在下載: 台灣大 各家新聞  進度: 30 / 68
目前正在下載: 台灣大 各家新聞  進度: 31 / 68
目前正在下載: 台灣大 各家新聞  進度: 32 / 68
目前正在下載: 台灣大 各家新聞  進度: 33 / 68
目前正在下載: 台灣大 各家新聞  進度: 34 / 68
目前正在下載: 台灣大 各家新聞  進度: 35 / 68
目前正在下載: 台灣大 各家新聞  進度: 36 / 68
目前正在下載: 台灣大 各家新聞  進度: 37 / 68
目前正在下載: 台灣大 各家新聞  進度: 38 / 68
目前正在下載: 台灣大 各家新聞  進度: 39 / 68
目前正在下載: 台灣大 各家新聞  進度: 40 / 68
目前正在下載: 台灣大 各家新聞  進度: 41 / 68
目前正在下載: 台灣大 各家新聞  進度: 42 / 68
目前正在下載: 台灣大 各家新聞  進度: 43 / 68
目前正在下載: 台灣大 各家新聞  進度: 44 / 68
目前正在下載: 台灣大 各家新聞  進度: 45 / 68
目前正在下載: 台灣大 各家新聞  進度: 46 / 68
目前正在下載: 台灣大 各家新聞  進度: 47 / 68
目前正在下載: 台灣大 各家新聞  進度: 48 / 68
目前正在下載: 台灣大 各家新聞  進度: 49 / 68
目前正在下載: 台灣

目前正在下載: 國巨 各家新聞  進度: 28 / 32
目前正在下載: 國巨 各家新聞  進度: 29 / 32
目前正在下載: 國巨 各家新聞  進度: 30 / 32
目前正在下載: 國巨 各家新聞  進度: 31 / 32
目前正在下載: 國巨 各家新聞  進度: 32 / 32
目前正在搜尋股票: 臺股期貨 在Google的新聞清單  進度: 17 / 20
目前正在下載: 臺股期貨 各家新聞  進度: 1 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 2 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 3 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 4 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 5 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 6 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 7 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 8 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 9 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 10 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 11 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 12 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 13 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 14 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 15 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 16 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 17 / 18
目前正在下載: 臺股期貨 各家新聞  進度: 18 / 18
目前正在搜尋股票: 友達 在Google的新聞清單  進度: 18 / 20
目前正在下載: 友達 各家新聞  進度: 1 / 32
目前正在下載: 友達 各家新聞  進度: 2 / 32
目前正在下載: 友達 各家新聞  進度: 3 / 32
目前正在下載: 友達 各家新聞  進度: 4 / 32
目前正在下載: 友達 各家新聞  進度: 5 / 32
目前正在下載: 友達 各家新聞  進度: 6 / 32
目前正在下載: 友達 各家新聞  進度: 7 / 32
目前正在下載: 友達 各家新聞  進度: 8 / 32
目前

In [10]:
searchList = searchList_all[2]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 瑞昱 在Google的新聞清單  進度: 1 / 20
目前正在下載: 瑞昱 各家新聞  進度: 1 / 27
目前正在下載: 瑞昱 各家新聞  進度: 2 / 27
目前正在下載: 瑞昱 各家新聞  進度: 3 / 27
目前正在下載: 瑞昱 各家新聞  進度: 4 / 27
目前正在下載: 瑞昱 各家新聞  進度: 5 / 27
目前正在下載: 瑞昱 各家新聞  進度: 6 / 27
目前正在下載: 瑞昱 各家新聞  進度: 7 / 27
目前正在下載: 瑞昱 各家新聞  進度: 8 / 27
目前正在下載: 瑞昱 各家新聞  進度: 9 / 27
目前正在下載: 瑞昱 各家新聞  進度: 10 / 27
目前正在下載: 瑞昱 各家新聞  進度: 11 / 27
目前正在下載: 瑞昱 各家新聞  進度: 12 / 27
目前正在下載: 瑞昱 各家新聞  進度: 13 / 27
目前正在下載: 瑞昱 各家新聞  進度: 14 / 27
目前正在下載: 瑞昱 各家新聞  進度: 15 / 27
目前正在下載: 瑞昱 各家新聞  進度: 16 / 27
目前正在下載: 瑞昱 各家新聞  進度: 17 / 27
目前正在下載: 瑞昱 各家新聞  進度: 18 / 27
目前正在下載: 瑞昱 各家新聞  進度: 19 / 27
目前正在下載: 瑞昱 各家新聞  進度: 20 / 27
目前正在下載: 瑞昱 各家新聞  進度: 21 / 27
目前正在下載: 瑞昱 各家新聞  進度: 22 / 27
目前正在下載: 瑞昱 各家新聞  進度: 23 / 27
目前正在下載: 瑞昱 各家新聞  進度: 24 / 27
目前正在下載: 瑞昱 各家新聞  進度: 25 / 27
目前正在下載: 瑞昱 各家新聞  進度: 26 / 27
目前正在下載: 瑞昱 各家新聞  進度: 27 / 27
目前正在搜尋股票: 亞德客-KY 在Google的新聞清單  進度: 2 / 20
目前正在下載: 亞德客-KY 各家新聞  進度: 1 / 9
目前正在下載: 亞德客-KY 各家新聞  進度: 2 / 9
目前正在下載: 亞德客-KY 各家新聞  進度: 3 / 9
目前正在下載: 亞德客-KY 各家新聞  進度: 4 / 9
目前正在下載: 亞德客-KY 各家新聞  進

目前正在下載: 華新 各家新聞  進度: 1 / 31
目前正在下載: 華新 各家新聞  進度: 2 / 31
目前正在下載: 華新 各家新聞  進度: 3 / 31
目前正在下載: 華新 各家新聞  進度: 4 / 31
目前正在下載: 華新 各家新聞  進度: 5 / 31
目前正在下載: 華新 各家新聞  進度: 6 / 31
目前正在下載: 華新 各家新聞  進度: 7 / 31
目前正在下載: 華新 各家新聞  進度: 8 / 31
目前正在下載: 華新 各家新聞  進度: 9 / 31
目前正在下載: 華新 各家新聞  進度: 10 / 31
目前正在下載: 華新 各家新聞  進度: 11 / 31
目前正在下載: 華新 各家新聞  進度: 12 / 31
目前正在下載: 華新 各家新聞  進度: 13 / 31
目前正在下載: 華新 各家新聞  進度: 14 / 31
目前正在下載: 華新 各家新聞  進度: 15 / 31
目前正在下載: 華新 各家新聞  進度: 16 / 31
目前正在下載: 華新 各家新聞  進度: 17 / 31
目前正在下載: 華新 各家新聞  進度: 18 / 31
目前正在下載: 華新 各家新聞  進度: 19 / 31
目前正在下載: 華新 各家新聞  進度: 20 / 31
目前正在下載: 華新 各家新聞  進度: 21 / 31
目前正在下載: 華新 各家新聞  進度: 22 / 31
目前正在下載: 華新 各家新聞  進度: 23 / 31
目前正在下載: 華新 各家新聞  進度: 24 / 31
目前正在下載: 華新 各家新聞  進度: 25 / 31
目前正在下載: 華新 各家新聞  進度: 26 / 31
目前正在下載: 華新 各家新聞  進度: 27 / 31
目前正在下載: 華新 各家新聞  進度: 28 / 31
目前正在下載: 華新 各家新聞  進度: 29 / 31
目前正在下載: 華新 各家新聞  進度: 30 / 31
目前正在下載: 華新 各家新聞  進度: 31 / 31
目前正在搜尋股票: 光寶科 在Google的新聞清單  進度: 13 / 20
目前正在下載: 光寶科 各家新聞  進度: 1 / 12
目前正在下載: 光寶科 各家新聞  進度: 2 / 12
目前正在下載: 光寶科 

目前正在下載: 仁寶 各家新聞  進度: 31 / 31


In [11]:
searchList = searchList_all[3]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 旭隼 在Google的新聞清單  進度: 1 / 20
目前正在下載: 旭隼 各家新聞  進度: 1 / 10
目前正在下載: 旭隼 各家新聞  進度: 2 / 10
目前正在下載: 旭隼 各家新聞  進度: 3 / 10
目前正在下載: 旭隼 各家新聞  進度: 4 / 10
目前正在下載: 旭隼 各家新聞  進度: 5 / 10
目前正在下載: 旭隼 各家新聞  進度: 6 / 10
目前正在下載: 旭隼 各家新聞  進度: 7 / 10
目前正在下載: 旭隼 各家新聞  進度: 8 / 10
目前正在下載: 旭隼 各家新聞  進度: 9 / 10
目前正在下載: 旭隼 各家新聞  進度: 10 / 10
目前正在搜尋股票: 力積電 在Google的新聞清單  進度: 2 / 20
目前正在下載: 力積電 各家新聞  進度: 1 / 29
目前正在下載: 力積電 各家新聞  進度: 2 / 29
目前正在下載: 力積電 各家新聞  進度: 3 / 29
目前正在下載: 力積電 各家新聞  進度: 4 / 29
目前正在下載: 力積電 各家新聞  進度: 5 / 29
目前正在下載: 力積電 各家新聞  進度: 6 / 29
目前正在下載: 力積電 各家新聞  進度: 7 / 29
目前正在下載: 力積電 各家新聞  進度: 8 / 29
目前正在下載: 力積電 各家新聞  進度: 9 / 29
目前正在下載: 力積電 各家新聞  進度: 10 / 29
目前正在下載: 力積電 各家新聞  進度: 11 / 29
目前正在下載: 力積電 各家新聞  進度: 12 / 29
目前正在下載: 力積電 各家新聞  進度: 13 / 29
目前正在下載: 力積電 各家新聞  進度: 14 / 29
目前正在下載: 力積電 各家新聞  進度: 15 / 29
目前正在下載: 力積電 各家新聞  進度: 16 / 29
目前正在下載: 力積電 各家新聞  進度: 17 / 29
目前正在下載: 力積電 各家新聞  進度: 18 / 29
目前正在下載: 力積電 各家新聞  進度: 19 / 29
目前正在下載: 力積電 各家新聞  進度: 20 / 29
目前正在下載: 力積電 各家新聞  進度: 21 / 29
目前正在下載: 力積電 各家新聞  進度:

目前正在下載: 大聯大 各家新聞  進度: 29 / 48
目前正在下載: 大聯大 各家新聞  進度: 30 / 48
目前正在下載: 大聯大 各家新聞  進度: 31 / 48
目前正在下載: 大聯大 各家新聞  進度: 32 / 48
目前正在下載: 大聯大 各家新聞  進度: 33 / 48
目前正在下載: 大聯大 各家新聞  進度: 34 / 48
目前正在下載: 大聯大 各家新聞  進度: 35 / 48
目前正在下載: 大聯大 各家新聞  進度: 36 / 48
目前正在下載: 大聯大 各家新聞  進度: 37 / 48
目前正在下載: 大聯大 各家新聞  進度: 38 / 48
目前正在下載: 大聯大 各家新聞  進度: 39 / 48
目前正在下載: 大聯大 各家新聞  進度: 40 / 48
目前正在下載: 大聯大 各家新聞  進度: 41 / 48
目前正在下載: 大聯大 各家新聞  進度: 42 / 48
目前正在下載: 大聯大 各家新聞  進度: 43 / 48
目前正在下載: 大聯大 各家新聞  進度: 44 / 48
目前正在下載: 大聯大 各家新聞  進度: 45 / 48
目前正在下載: 大聯大 各家新聞  進度: 46 / 48
目前正在下載: 大聯大 各家新聞  進度: 47 / 48
目前正在下載: 大聯大 各家新聞  進度: 48 / 48
目前正在搜尋股票: 臻鼎-KY 在Google的新聞清單  進度: 12 / 20
目前正在下載: 臻鼎-KY 各家新聞  進度: 1 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 2 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 3 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 4 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 5 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 6 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 7 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 8 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 9 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 10 / 12
目前正在下載: 臻鼎-KY 各家新聞  進度: 11 / 12
目前正在下載: 臻鼎-KY 各

目前正在下載: 台灣高鐵 各家新聞  進度: 28 / 31
目前正在下載: 台灣高鐵 各家新聞  進度: 29 / 31
目前正在下載: 台灣高鐵 各家新聞  進度: 30 / 31
目前正在下載: 台灣高鐵 各家新聞  進度: 31 / 31
目前正在搜尋股票: 華邦電 在Google的新聞清單  進度: 20 / 20
目前正在下載: 華邦電 各家新聞  進度: 1 / 16
目前正在下載: 華邦電 各家新聞  進度: 2 / 16
目前正在下載: 華邦電 各家新聞  進度: 3 / 16
目前正在下載: 華邦電 各家新聞  進度: 4 / 16
目前正在下載: 華邦電 各家新聞  進度: 5 / 16
目前正在下載: 華邦電 各家新聞  進度: 6 / 16
目前正在下載: 華邦電 各家新聞  進度: 7 / 16
目前正在下載: 華邦電 各家新聞  進度: 8 / 16
目前正在下載: 華邦電 各家新聞  進度: 9 / 16
目前正在下載: 華邦電 各家新聞  進度: 10 / 16
目前正在下載: 華邦電 各家新聞  進度: 11 / 16
目前正在下載: 華邦電 各家新聞  進度: 12 / 16
目前正在下載: 華邦電 各家新聞  進度: 13 / 16
目前正在下載: 華邦電 各家新聞  進度: 14 / 16
目前正在下載: 華邦電 各家新聞  進度: 15 / 16
目前正在下載: 華邦電 各家新聞  進度: 16 / 16


In [12]:
searchList = searchList_all[4]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 旺宏 在Google的新聞清單  進度: 1 / 20
目前正在下載: 旺宏 各家新聞  進度: 1 / 23
目前正在下載: 旺宏 各家新聞  進度: 2 / 23
目前正在下載: 旺宏 各家新聞  進度: 3 / 23
目前正在下載: 旺宏 各家新聞  進度: 4 / 23
目前正在下載: 旺宏 各家新聞  進度: 5 / 23
目前正在下載: 旺宏 各家新聞  進度: 6 / 23
目前正在下載: 旺宏 各家新聞  進度: 7 / 23
目前正在下載: 旺宏 各家新聞  進度: 8 / 23
目前正在下載: 旺宏 各家新聞  進度: 9 / 23
目前正在下載: 旺宏 各家新聞  進度: 10 / 23
目前正在下載: 旺宏 各家新聞  進度: 11 / 23
目前正在下載: 旺宏 各家新聞  進度: 12 / 23
目前正在下載: 旺宏 各家新聞  進度: 13 / 23
目前正在下載: 旺宏 各家新聞  進度: 14 / 23
目前正在下載: 旺宏 各家新聞  進度: 15 / 23
目前正在下載: 旺宏 各家新聞  進度: 16 / 23
目前正在下載: 旺宏 各家新聞  進度: 17 / 23
目前正在下載: 旺宏 各家新聞  進度: 18 / 23
目前正在下載: 旺宏 各家新聞  進度: 19 / 23
目前正在下載: 旺宏 各家新聞  進度: 20 / 23
目前正在下載: 旺宏 各家新聞  進度: 21 / 23
目前正在下載: 旺宏 各家新聞  進度: 22 / 23
目前正在下載: 旺宏 各家新聞  進度: 23 / 23
目前正在搜尋股票: 華航 在Google的新聞清單  進度: 2 / 20
目前正在下載: 華航 各家新聞  進度: 1 / 27
目前正在下載: 華航 各家新聞  進度: 2 / 27
目前正在下載: 華航 各家新聞  進度: 3 / 27
目前正在下載: 華航 各家新聞  進度: 4 / 27
目前正在下載: 華航 各家新聞  進度: 5 / 27
目前正在下載: 華航 各家新聞  進度: 6 / 27
目前正在下載: 華航 各家新聞  進度: 7 / 27
目前正在下載: 華航 各家新聞  進度: 8 / 27
目前正在下載: 華航 各家新聞  進度: 9 / 27
目前正在下載: 華航 各家新

目前正在下載: 華通 各家新聞  進度: 20 / 23
目前正在下載: 華通 各家新聞  進度: 21 / 23
目前正在下載: 華通 各家新聞  進度: 22 / 23
目前正在下載: 華通 各家新聞  進度: 23 / 23
目前正在搜尋股票: 創意 在Google的新聞清單  進度: 12 / 20
目前正在下載: 創意 各家新聞  進度: 1 / 68
目前正在下載: 創意 各家新聞  進度: 2 / 68
目前正在下載: 創意 各家新聞  進度: 3 / 68
目前正在下載: 創意 各家新聞  進度: 4 / 68
目前正在下載: 創意 各家新聞  進度: 5 / 68
目前正在下載: 創意 各家新聞  進度: 6 / 68
目前正在下載: 創意 各家新聞  進度: 7 / 68
目前正在下載: 創意 各家新聞  進度: 8 / 68
目前正在下載: 創意 各家新聞  進度: 9 / 68
目前正在下載: 創意 各家新聞  進度: 10 / 68
目前正在下載: 創意 各家新聞  進度: 11 / 68
目前正在下載: 創意 各家新聞  進度: 12 / 68
目前正在下載: 創意 各家新聞  進度: 13 / 68
目前正在下載: 創意 各家新聞  進度: 14 / 68
目前正在下載: 創意 各家新聞  進度: 15 / 68
目前正在下載: 創意 各家新聞  進度: 16 / 68
目前正在下載: 創意 各家新聞  進度: 17 / 68
目前正在下載: 創意 各家新聞  進度: 18 / 68
目前正在下載: 創意 各家新聞  進度: 19 / 68
目前正在下載: 創意 各家新聞  進度: 20 / 68
目前正在下載: 創意 各家新聞  進度: 21 / 68
目前正在下載: 創意 各家新聞  進度: 22 / 68
目前正在下載: 創意 各家新聞  進度: 23 / 68
目前正在下載: 創意 各家新聞  進度: 24 / 68
目前正在下載: 創意 各家新聞  進度: 25 / 68
目前正在下載: 創意 各家新聞  進度: 26 / 68
目前正在下載: 創意 各家新聞  進度: 27 / 68
目前正在下載: 創意 各家新聞  進度: 28 / 68
目前正在下載: 創意 各家新聞  進度: 29 / 68
目前正在下載: 創意 各家

In [13]:
searchList = searchList_all[5]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 寶成 在Google的新聞清單  進度: 1 / 20
目前正在下載: 寶成 各家新聞  進度: 1 / 36
目前正在下載: 寶成 各家新聞  進度: 2 / 36
目前正在下載: 寶成 各家新聞  進度: 3 / 36
目前正在下載: 寶成 各家新聞  進度: 4 / 36
目前正在下載: 寶成 各家新聞  進度: 5 / 36
目前正在下載: 寶成 各家新聞  進度: 6 / 36
目前正在下載: 寶成 各家新聞  進度: 7 / 36
目前正在下載: 寶成 各家新聞  進度: 8 / 36
目前正在下載: 寶成 各家新聞  進度: 9 / 36
目前正在下載: 寶成 各家新聞  進度: 10 / 36
目前正在下載: 寶成 各家新聞  進度: 11 / 36
目前正在下載: 寶成 各家新聞  進度: 12 / 36
目前正在下載: 寶成 各家新聞  進度: 13 / 36
目前正在下載: 寶成 各家新聞  進度: 14 / 36
目前正在下載: 寶成 各家新聞  進度: 15 / 36
目前正在下載: 寶成 各家新聞  進度: 16 / 36
目前正在下載: 寶成 各家新聞  進度: 17 / 36
目前正在下載: 寶成 各家新聞  進度: 18 / 36
目前正在下載: 寶成 各家新聞  進度: 19 / 36
目前正在下載: 寶成 各家新聞  進度: 20 / 36
目前正在下載: 寶成 各家新聞  進度: 21 / 36
目前正在下載: 寶成 各家新聞  進度: 22 / 36
目前正在下載: 寶成 各家新聞  進度: 23 / 36
目前正在下載: 寶成 各家新聞  進度: 24 / 36
目前正在下載: 寶成 各家新聞  進度: 25 / 36
目前正在下載: 寶成 各家新聞  進度: 26 / 36
目前正在下載: 寶成 各家新聞  進度: 27 / 36
目前正在下載: 寶成 各家新聞  進度: 28 / 36
目前正在下載: 寶成 各家新聞  進度: 29 / 36
目前正在下載: 寶成 各家新聞  進度: 30 / 36
目前正在下載: 寶成 各家新聞  進度: 31 / 36
目前正在下載: 寶成 各家新聞  進度: 32 / 36
目前正在下載: 寶成 各家新聞  進度: 33 / 36
目前正在下載: 寶成 各家新

目前正在下載: 大成 各家新聞  進度: 26 / 77
目前正在下載: 大成 各家新聞  進度: 27 / 77
目前正在下載: 大成 各家新聞  進度: 28 / 77
目前正在下載: 大成 各家新聞  進度: 29 / 77
目前正在下載: 大成 各家新聞  進度: 30 / 77
目前正在下載: 大成 各家新聞  進度: 31 / 77
目前正在下載: 大成 各家新聞  進度: 32 / 77
目前正在下載: 大成 各家新聞  進度: 33 / 77
目前正在下載: 大成 各家新聞  進度: 34 / 77
目前正在下載: 大成 各家新聞  進度: 35 / 77
目前正在下載: 大成 各家新聞  進度: 36 / 77
目前正在下載: 大成 各家新聞  進度: 37 / 77
目前正在下載: 大成 各家新聞  進度: 38 / 77
目前正在下載: 大成 各家新聞  進度: 39 / 77
目前正在下載: 大成 各家新聞  進度: 40 / 77
目前正在下載: 大成 各家新聞  進度: 41 / 77
目前正在下載: 大成 各家新聞  進度: 42 / 77
目前正在下載: 大成 各家新聞  進度: 43 / 77
目前正在下載: 大成 各家新聞  進度: 44 / 77
目前正在下載: 大成 各家新聞  進度: 45 / 77
目前正在下載: 大成 各家新聞  進度: 46 / 77
目前正在下載: 大成 各家新聞  進度: 47 / 77
目前正在下載: 大成 各家新聞  進度: 48 / 77
目前正在下載: 大成 各家新聞  進度: 49 / 77
目前正在下載: 大成 各家新聞  進度: 50 / 77
目前正在下載: 大成 各家新聞  進度: 51 / 77
目前正在下載: 大成 各家新聞  進度: 52 / 77
目前正在下載: 大成 各家新聞  進度: 53 / 77
目前正在下載: 大成 各家新聞  進度: 54 / 77
目前正在下載: 大成 各家新聞  進度: 55 / 77
目前正在下載: 大成 各家新聞  進度: 56 / 77
目前正在下載: 大成 各家新聞  進度: 57 / 77
目前正在下載: 大成 各家新聞  進度: 58 / 77
目前正在下載: 大成 各家新聞  進度: 59 / 77
目前正在下載: 大成 各家新

In [14]:
searchList = searchList_all[6]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 智原 在Google的新聞清單  進度: 1 / 20
目前正在下載: 智原 各家新聞  進度: 1 / 16
目前正在下載: 智原 各家新聞  進度: 2 / 16
目前正在下載: 智原 各家新聞  進度: 3 / 16
目前正在下載: 智原 各家新聞  進度: 4 / 16
目前正在下載: 智原 各家新聞  進度: 5 / 16
目前正在下載: 智原 各家新聞  進度: 6 / 16
目前正在下載: 智原 各家新聞  進度: 7 / 16
目前正在下載: 智原 各家新聞  進度: 8 / 16
目前正在下載: 智原 各家新聞  進度: 9 / 16
目前正在下載: 智原 各家新聞  進度: 10 / 16
目前正在下載: 智原 各家新聞  進度: 11 / 16
目前正在下載: 智原 各家新聞  進度: 12 / 16
目前正在下載: 智原 各家新聞  進度: 13 / 16
目前正在下載: 智原 各家新聞  進度: 14 / 16
目前正在下載: 智原 各家新聞  進度: 15 / 16
目前正在下載: 智原 各家新聞  進度: 16 / 16
目前正在搜尋股票: 台玻 在Google的新聞清單  進度: 2 / 20
目前正在下載: 台玻 各家新聞  進度: 1 / 33
目前正在下載: 台玻 各家新聞  進度: 2 / 33
目前正在下載: 台玻 各家新聞  進度: 3 / 33
目前正在下載: 台玻 各家新聞  進度: 4 / 33
目前正在下載: 台玻 各家新聞  進度: 5 / 33
目前正在下載: 台玻 各家新聞  進度: 6 / 33
目前正在下載: 台玻 各家新聞  進度: 7 / 33
目前正在下載: 台玻 各家新聞  進度: 8 / 33
目前正在下載: 台玻 各家新聞  進度: 9 / 33
目前正在下載: 台玻 各家新聞  進度: 10 / 33
目前正在下載: 台玻 各家新聞  進度: 11 / 33
目前正在下載: 台玻 各家新聞  進度: 12 / 33
目前正在下載: 台玻 各家新聞  進度: 13 / 33
目前正在下載: 台玻 各家新聞  進度: 14 / 33
目前正在下載: 台玻 各家新聞  進度: 15 / 33
目前正在下載: 台玻 各家新聞  進度: 16 / 33
目前正在下載: 台玻 各家新

目前正在下載: 中鴻 各家新聞  進度: 15 / 42
目前正在下載: 中鴻 各家新聞  進度: 16 / 42
目前正在下載: 中鴻 各家新聞  進度: 17 / 42
目前正在下載: 中鴻 各家新聞  進度: 18 / 42
目前正在下載: 中鴻 各家新聞  進度: 19 / 42
目前正在下載: 中鴻 各家新聞  進度: 20 / 42
目前正在下載: 中鴻 各家新聞  進度: 21 / 42
目前正在下載: 中鴻 各家新聞  進度: 22 / 42
目前正在下載: 中鴻 各家新聞  進度: 23 / 42
目前正在下載: 中鴻 各家新聞  進度: 24 / 42
目前正在下載: 中鴻 各家新聞  進度: 25 / 42
目前正在下載: 中鴻 各家新聞  進度: 26 / 42
目前正在下載: 中鴻 各家新聞  進度: 27 / 42
目前正在下載: 中鴻 各家新聞  進度: 28 / 42
目前正在下載: 中鴻 各家新聞  進度: 29 / 42
目前正在下載: 中鴻 各家新聞  進度: 30 / 42
目前正在下載: 中鴻 各家新聞  進度: 31 / 42
目前正在下載: 中鴻 各家新聞  進度: 32 / 42
目前正在下載: 中鴻 各家新聞  進度: 33 / 42
目前正在下載: 中鴻 各家新聞  進度: 34 / 42
目前正在下載: 中鴻 各家新聞  進度: 35 / 42
目前正在下載: 中鴻 各家新聞  進度: 36 / 42
目前正在下載: 中鴻 各家新聞  進度: 37 / 42
目前正在下載: 中鴻 各家新聞  進度: 38 / 42
目前正在下載: 中鴻 各家新聞  進度: 39 / 42
目前正在下載: 中鴻 各家新聞  進度: 40 / 42
目前正在下載: 中鴻 各家新聞  進度: 41 / 42
目前正在下載: 中鴻 各家新聞  進度: 42 / 42
目前正在搜尋股票: 新唐 在Google的新聞清單  進度: 18 / 20
目前正在下載: 新唐 各家新聞  進度: 1 / 28
目前正在下載: 新唐 各家新聞  進度: 2 / 28
目前正在下載: 新唐 各家新聞  進度: 3 / 28
目前正在下載: 新唐 各家新聞  進度: 4 / 28
目前正在下載: 新唐 各家新聞  進度: 5 / 28
目前正在下載: 新

In [15]:
searchList = searchList_all[7]

for iSearch in range(len(searchList)):
    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 和潤企業 在Google的新聞清單  進度: 1 / 10
目前正在下載: 和潤企業 各家新聞  進度: 1 / 22
目前正在下載: 和潤企業 各家新聞  進度: 2 / 22
目前正在下載: 和潤企業 各家新聞  進度: 3 / 22
目前正在下載: 和潤企業 各家新聞  進度: 4 / 22
目前正在下載: 和潤企業 各家新聞  進度: 5 / 22
目前正在下載: 和潤企業 各家新聞  進度: 6 / 22
目前正在下載: 和潤企業 各家新聞  進度: 7 / 22
目前正在下載: 和潤企業 各家新聞  進度: 8 / 22
目前正在下載: 和潤企業 各家新聞  進度: 9 / 22
目前正在下載: 和潤企業 各家新聞  進度: 10 / 22
目前正在下載: 和潤企業 各家新聞  進度: 11 / 22
目前正在下載: 和潤企業 各家新聞  進度: 12 / 22
目前正在下載: 和潤企業 各家新聞  進度: 13 / 22
目前正在下載: 和潤企業 各家新聞  進度: 14 / 22
目前正在下載: 和潤企業 各家新聞  進度: 15 / 22
目前正在下載: 和潤企業 各家新聞  進度: 16 / 22
目前正在下載: 和潤企業 各家新聞  進度: 17 / 22
目前正在下載: 和潤企業 各家新聞  進度: 18 / 22
目前正在下載: 和潤企業 各家新聞  進度: 19 / 22
目前正在下載: 和潤企業 各家新聞  進度: 20 / 22
目前正在下載: 和潤企業 各家新聞  進度: 21 / 22
目前正在下載: 和潤企業 各家新聞  進度: 22 / 22
目前正在搜尋股票: 裕民 在Google的新聞清單  進度: 2 / 10
目前正在下載: 裕民 各家新聞  進度: 1 / 26
目前正在下載: 裕民 各家新聞  進度: 2 / 26
目前正在下載: 裕民 各家新聞  進度: 3 / 26
目前正在下載: 裕民 各家新聞  進度: 4 / 26
目前正在下載: 裕民 各家新聞  進度: 5 / 26
目前正在下載: 裕民 各家新聞  進度: 6 / 26
目前正在下載: 裕民 各家新聞  進度: 7 / 26
目前正在下載: 裕民 各家新聞  進度: 8 / 26
目前正在下載: 裕民 各家新聞  進度: 9 / 

In [16]:
filtered_stockNews = stockNews.query('content != "unknow domain"').reset_index(drop=True)

print("Old data frame length:", len(stockNews), "\nNew data frame length:", 
       len(filtered_stockNews), "\nNumber of rows with unknown domain: ",
       (len(stockNews)-len(filtered_stockNews)))

Old data frame length: 3852 
New data frame length: 2691 
Number of rows with unknown domain:  1161


In [17]:
path = 'C:/Users/USER/Desktop/Prof_Hsieh_Project/about_emotion/Chinese_derivative/corpus'
os.chdir(path)

filename = StartDate + "_NewsData.pkl"
filtered_stockNews.to_pickle(filename)